In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from datetime import timedelta
from datetime import datetime
from datetime import timezone
import time
import sqlalchemy

In [47]:
import io
from sqlalchemy import create_engine
import psycopg2 as pg
#load python script that batch loads pandas df to sql
from io import StringIO

In [2]:
pip install sodapy

Note: you may need to restart the kernel to use updated packages.


In [3]:
from sodapy import Socrata
from config import MyAppToken
from config import username
from config import password

In [4]:
engine = create_engine("sqlite:///final_project")

In [7]:
#load in the data via API call
client = Socrata('data.austintexas.gov', MyAppToken, username, password)

In [8]:
results = client.get("xwdj-i9he", limit=1160000)

In [9]:
results_df = pd.DataFrame.from_records(results)

In [10]:
results_df.sample(25)

,sr_location,sr_location_council_district,sr_method_received_desc,sr_closed_date,sr_location_x,sr_status_desc,sr_type_desc,sr_location_county,sr_location_map_page,sr_location_long,...,sr_location_lat,sr_location_lat_long,sr_status_date,sr_location_street_name,sr_created_date,sr_location_city,sr_location_y,sr_location_map_tile,sr_location_zip_code,sr_location_street_number
530602,"4709 CANDLETREE LN, AUSTIN, TX 78744",2,Phone,2018-10-24T13:38:09.000,3113741.99697997,Closed,ARR Missed Recycling,TRAVIS,675A,-97.74673768,...,30.18887143,"{'latitude': '30.18887143', 'needs_recoding': ...",2018-10-24T13:38:09.000,CANDLETREE,2018-08-20T14:01:46.000,AUSTIN,10041930.0014338,MJ15,78744,4709
425230,"300 E RIVERSIDE DR, AUSTIN, TX 78704",9,Spot311 Interface,2019-04-19T19:56:37.000,3114235.43491865,Closed,Dockless Mobility,TRAVIS,615E,-97.74341325,...,30.25461477,"{'latitude': '30.25461477', 'needs_recoding': ...",2019-04-19T19:56:37.000,RIVERSIDE,2019-04-19T18:17:58.000,AUSTIN,10065858.2728743,MJ21,78704,300
397523,"520 CARISMATIC LN, AUSTIN, TX 78748",5,Phone,2019-07-25T07:54:20.000,3093406.40151796,Closed,ARR Brush and Bulk,TRAVIS,703C,-97.81210427,...,30.15108153,"{'latitude': '30.15108153', 'needs_recoding': ...",2019-07-25T07:54:20.000,CARISMATIC,2019-06-07T13:04:14.000,AUSTIN,10027715.48211,MF12,78748,520
558122,"1032 PAYTON GIN RD, AUSTIN, TX 78758",4,Phone,2018-06-21T07:25:38.000,3124992.61997749,Closed,Austin Code - Request Code Officer,TRAVIS,526E,-97.706442,...,30.36089594,"{'latitude': '30.36089594', 'needs_recoding': ...",2018-06-21T07:25:38.000,PAYTON GIN,2018-06-21T07:15:23.000,AUSTIN,10104774.5495392,ML30,78758,1032
194225,"2223 SOUTH LAKESHORE BLVD, AUSTIN, TX 78741",3,Spot311 Interface,2020-08-22T14:19:26.000,3121805.66228940,Closed,Parking Violation Enforcement,TRAVIS,615Q,-97.7197383,...,30.24322857,"{'latitude': '30.24322857', 'human_address': '...",2020-08-22T14:19:26.000,SOUTH LAKESHORE,2020-08-22T13:24:01.000,AUSTIN,10061893.16955990,MK20,78741,2223
267940,"10614 SANS SOUCI PL, AUSTIN, TX 78759",10,Phone,2020-03-04T07:32:01.000,3105783.50844373,Closed,Loose Dog,TRAVIS,464U,-97.76598872,...,30.41260565,"{'latitude': '30.41260565', 'human_address': '...",2020-03-04T07:32:01.000,SANS SOUCI,2020-03-03T12:05:08.000,AUSTIN,10123137.34151120,MH34,78759,10614
995510,"6615 CRUZ ST, AUSTIN, TX",3,Phone,2015-04-02T15:58:57.000,3128819.73528324,Closed,Animal Control - Assistance Request,TRAVIS,616X,-97.69794015,...,30.22781939,"{'latitude': '30.22781939', 'needs_recoding': ...",2015-04-02T15:58:57.000,CRUZ,2015-04-02T10:11:50.000,AUSTIN,10056452.9340832,ML19,78741,6615
1027464,"8509 FM 969 RD, AUSTIN, TX",1,Phone,2014-12-16T13:56:08.000,3147469.48692161,Closed,School Zone - New/Review/Changes,TRAVIS,587Q,-97.63750223,...,30.27688094,"{'latitude': '30.27688094', 'needs_recoding': ...",2014-12-16T13:56:08.000,969,2014-12-16T11:24:35.000,AUSTIN,10074748.2786151,MP23,78725,8509
88773,"1112 W PARMER LN, AUSTIN, TX 78753",7,Mobile Device,2021-04-07T11:57:40.000,3133159.18178302,Closed,Austin Code - Request Code Officer,TRAVIS,466Y,-97.67924309,...,30.40805774,"{'latitude': '30.40805774', 'human_address': '...",2021-04-07T11:57:40.000,PARMER,2021-04-07T11:47:52.000,AUSTIN,10122125.46279420,MM34,78753,1112
416556,"8312 N IH 35 SB 23, AUSTIN, TX 78753",4,Web,2019-05-05T17:58:18.000,3128362.98099741,Closed,Austin Code - Request Code Officer,TRAVIS,526P,-97.69616604,...,30.34577373,"{'latitude': '30.34577373', 'needs_recoding': ...",2019-05-05T17:58:18.000,IH 35,2019-05-05T17:39:13.000,AUSTIN,10099352.9876463,ML29,78753,8312


In [11]:
len(results_df)

1151435

In [12]:
results_df["Created Date"] = pd.to_datetime(results_df["sr_created_date"])
results_df['Closed Date'] = pd.to_datetime(results_df['sr_closed_date'])

In [13]:
results_df['Request Year'] = pd.DatetimeIndex(results_df['Created Date']).year

In [14]:
#filter the df by Request Year
results_df = results_df[(results_df['Request Year'] == 2018) | (results_df['Request Year'] == 2019)]

In [15]:
results_df['Open Time'] = (results_df['Closed Date'] - results_df['Created Date'])

In [16]:
results_df['Open Time delta'] = pd.to_timedelta(results_df['Open Time'])
results_df['Total Seconds'] = (results_df['Open Time delta']).dt.total_seconds()

In [ ]:
results_df = results_df[(results_df['Total Seconds'] < 31540000)]

In [25]:
median = results_df['Total Seconds'].median(axis=0)

In [26]:
#median

69911.5

In [27]:
conditions = [
    (results_df['Total Seconds'] <= median),
    (results_df['Total Seconds'] > median)
    ]
values = ['Quick Response', 'Slow Response']
results_df['Response Time'] = np.select(conditions, values)

In [20]:
#print(results_df['Total Seconds'].max())

109367883.0


In [21]:
#results_df.loc[results_df['Total Seconds'] == 109367883]

,sr_location,sr_location_council_district,sr_method_received_desc,sr_closed_date,sr_location_x,sr_status_desc,sr_type_desc,sr_location_county,sr_location_map_page,sr_location_long,...,sr_location_map_tile,sr_location_zip_code,sr_location_street_number,Created Date,Closed Date,Request Year,Open Time,Open Time delta,Total Seconds,Response Time
610168,"1520 RUTHERFORD LN, AUSTIN, TX 78754",1,Phone,2021-08-05T14:23:43.000,3133220.45539295,Closed,Animal Control - Assistance Request,TRAVIS,526Y,-97.68097996,...,MM28,78754,1520,2018-02-16 18:25:40,2021-08-05 14:23:43,2018,1265 days 19:58:03,1265 days 19:58:03,109367883.0,Slow Response


In [22]:
#np.nanpercentile(results_df['Total Seconds'], 99.35) 

31898303.480000004

In [28]:
response = results_df['Response Time'].value_counts()
response

Quick Response    163163
Slow Response     163163
Name: Response Time, dtype: int64

In [29]:
code_types = ['Austin Code - Request Code Officer', 'Code Compliance', 'Code Compliance', 'Construction Ordinance SR', 'Austin Code - Signs/Billboards', 'Austin Code - Short Term Rental Complaint SR', 'Loud Commercial Music', 'Loud Music', 'Austin Code - Coronavirus', 'Austin Code - Short Term Rental (STR) Appointment', 'Austin Code - Construction Rest Break Complaint', 'Austin Code - Construction Ordinance SR', 'Short Term Rental Complaint SR',  'Austin Code - Woodridge Apt']
animal_types = ['Animal - Proper Care', 'Dangerous Animal - Except Dogs', 'Animal Bite',  'Animal Control - Assistance Request', 'Loose Dog', 'Found Animal - Pick Up', 'Loose Animal Not Dog', 'Injured / Sick Animal',  'Found Animal Report - Keep',  'Wildlife Exposure', 'Animal In Vehicle', 'Dangerous/Vicious Dog Investigation', 'Coyote Complaints', 'Animal Trapped in Storm Drain', 'Animal Roadside Sales', 'Loose Animal (not dog)', 'Bat Complaint']
public_types = ['Pothole Repair', 'Obstruction - Urban Forestry', 'Guardrail - New', 'Sidewalk Repair', 'Traffic Signal - Maintenance', 'Traffic Signal - Dig Tess Request',  'Street Light Issue- Address', 'Street Lights New', 'Pavement Failure', 'Sidewalk/Curb Ramp/Route - NEW',  'Alley & Unpaved Street Maintenance',  'Curb/Gutter Repair', 'Bridge Repair', 'Guardrail New/Repair', 'Street Light Issue- Multiple poles/multiple streets', 'Fence/Wall Repair' 'Street Resurfacing', 'Tree Issue ROW/Emergency (PW)', 'Tree Issue ROW/Maintenance (PW)', 'Street Resurfacing Inquiry', 'Guardrail Repair', 'Barricade Request', 'Road Sanding Request']
resource_types = ['ARR Dead Animal Collection', 'ARR Missed Recycling', 'Debris in Street', 'ARR Missed Recycling',  'ARR Missed Yard Trimmings/Compost', 'ARR Brush and Bulk', 'ARR Street Sweeping', 'ARR Missed Yard Trimmings /Organics',  'Ordinance Universal Recycling',  'Ordinance Single-Use Carryout Bags', 'ARR Dumpster', 'Roadway Spillage']
transport_types = ['Lane/Road Closure Notification', 'Traffic Engineering - Jurisdiction Issue', 'Sign - New', 'Bicycle Issues', 'Traffic Calming', 'Traffic Signal - New/Change', 'Shared Micromobility', 'Sign - New',  'Sign - Traffic Sign Emergency', 'Traffic Engineering - General', 'Concerns in the ROW', 'Obstruction in ROW', 'Construction/Permitting- ROW', 'Tree Issue ROW', 'Sign - Traffic Sign Maintenance', 'Road Markings/Striping - Maintenance', 'Parking Violation Enforcement', 'Construction Items in ROW', 'Parking Machine Issue', 'Speed Management', 'Speed Limit - Changes/Signs', 'View Obstruction at Intersection', 'Sign - School Zone Sign Maintenance', 'Sign - Street Name', 'Sign - Parking Sign Maintenance', 'Road Markings/Striping - New', 'School Zone Flasher - Timing/Maintenance', 'ATD - Booting Complaint', 'Filming Permit - ROW', 'Residential Parking Permit Enforcement', 'School Zone - New/Review/Changes', 'EV PIE Station Issues', 'Valet Permit - ROW', 'Vendor Permit - ROW', 'Parking Permit- ROW', 'ATD - Shared Micromobility', 'Newspaper Rack- ROW', 'Construction items in ROW', 'EV PIP Residential Rebate', 'ATD - Pay-by-Phone App', 'Parking Ticket Complaint', 'School Issues - Crossing Guards', 'Dockless Mobility', 'Construction Items- ROW', 'Guardrail - New', 'Emergency Road Closure Report', 'Mowing Medians', 'Emergency Vehicle Preemption Device']
watershed_types = ['Flooding  Current (Non-Emergency)', 'Flood Report', 'Flooding - Storms During Business Hours', 'Flooding - Storms After Business Hours', 'Channels/Creeks/Drainage Easement', 'Flooding - Past', 'Creek & Pond Vegetation Control', 'Storm Drain Pipe Services', 'Standing Water', 'Ditch/Driveway Pipe Services', 'Flooding Current (Non-Emergency)', 'Drainage - Miscellaneous', 'Town Lake Debris Issues', 'Lost Item in Storm Drainage System']
park_types = ['Park Maintenance - Grounds', 'Park Maintenance - Grounds Plumbing Issues', 'Park Maintenance - Pool Issues', 'Park Maintenance - Cemeteries', 'Mowing City Parks', 'Park Maintenance - Building Issues', 'Park Maintenance - Grounds Electrical Issues', 'Park Maintenance - Building Plumbing Issues', 'Park Maintenance - Building Electrical Issues', 'Park Maintenance - Building A/C & Heating Issues', 'Aquatics Hotline Inquiry']
water_types = ['AW - Water Theft Report', 'AW - Water Waste Report', 'Water Waste Report', 'Water Theft Report']
health_types = ['Public Health - Graffiti Abatement', 'Basic Needs - Appointment', 'Basic Needs - Information Referral',  'Public Health - Environmental Services - City', 'Public Health - Environmental Services - County', 'Graffiti Abatement', 'Public Health - Environmental Services - City', 'Heat Advisory', 'HSEM Heat Advisory']
housing_types = ['LeadSmart Program', 'Neighborhood Home Programs']
fire_types = ['AFD - Wildfire Concern / Presentation']

In [30]:
results_df['Code Check'] = results_df['sr_type_desc'].isin(code_types)
results_df['Animal Check'] = results_df['sr_type_desc'].isin(animal_types)
results_df['Public Types'] = results_df['sr_type_desc'].isin(public_types)
results_df['Resource Check'] = results_df['sr_type_desc'].isin(resource_types)
results_df['Transport Check'] = results_df['sr_type_desc'].isin(transport_types)                                                                
results_df['Watershed Check'] = results_df['sr_type_desc'].isin(watershed_types)
results_df['Park Check'] = results_df['sr_type_desc'].isin(park_types)
results_df['Water Check'] = results_df['sr_type_desc'].isin(water_types)
results_df['Health Check'] = results_df['sr_type_desc'].isin(health_types)
results_df['Housing Check'] = results_df['sr_type_desc'].isin(housing_types)
results_df['Fire Check'] = results_df['sr_type_desc'].isin(fire_types)

In [31]:
results_df.loc[results_df["Code Check"] == True, "Department"] = 'Austin Code'
results_df.loc[results_df["Animal Check"] == True, "Department"] = 'Animal Services'
results_df.loc[results_df["Public Types"] == True, "Department"] = 'Public Works'
results_df.loc[results_df["Resource Check"] == True, "Department"] = 'Austin Resource Recovery'
results_df.loc[results_df["Transport Check"] == True, "Department"] = 'Austin Transportation'
results_df.loc[results_df["Watershed Check"] == True, "Department"] = 'Watershed Check'
results_df.loc[results_df["Park Check"] == True, "Department"] = 'Parks and Recreation'
results_df.loc[results_df["Water Check"] == True, "Department"] = 'Austin Water'
results_df.loc[results_df["Health Check"] == True, "Department"] = 'Public Health'
results_df.loc[results_df["Housing Check"] == True, "Department"] = 'Housing and Planning'
results_df.loc[results_df["Fire Check"] == True, "Department"] = 'Austin Fire'

In [32]:
#top_dept = results_df['Department'].value_counts()
#top_dept

Austin Code                 75872
Animal Services             60761
Austin Transportation       56381
Public Works                56162
Austin Resource Recovery    50856
Parks and Recreation         8728
Public Health                7979
Watershed Check              4887
Austin Water                 4058
Housing and Planning           87
Austin Fire                    65
Name: Department, dtype: int64

In [98]:
#print(results_df.columns)

Index(['sr_location', 'sr_location_council_district',
       'sr_method_received_desc', 'sr_closed_date', 'sr_location_x',
       'sr_status_desc', 'sr_type_desc', 'sr_location_county',
       'sr_location_map_page', 'sr_location_long', 'sr_updated_date',
       'sr_number', 'sr_location_lat', 'sr_location_lat_long',
       'sr_status_date', 'sr_location_street_name', 'sr_created_date',
       'sr_location_street_number', 'sr_location_city', 'sr_location_y',
       'sr_location_map_tile', 'sr_location_zip_code', 'Created Date',
       'Closed Date', 'Request Year', 'Open Time', 'Open Time delta',
       'Total Seconds', 'Response Time', 'Department'],
      dtype='object')


In [33]:
results_df = results_df.drop(columns=['Code Check', 'Animal Check', 'Public Types','Resource Check','Transport Check','Watershed Check','Park Check','Water Check','Health Check','Housing Check','Fire Check'])

In [54]:
training_df = results_df.drop(columns=['sr_location', 'Open Time', 'sr_location_lat_long', 'sr_location_council_district', 'sr_closed_date', 'sr_location_x', 'sr_status_desc', 'sr_location_county', 'sr_location_map_page', 'sr_location_long', 'sr_updated_date', 'sr_location_lat', 'sr_status_date', 'sr_location_street_name', 'sr_created_date', 'sr_location_street_number', 'sr_location_city', 'sr_location_y', 'sr_location_map_tile', 'Created Date', 'Closed Date', 'Open Time delta'])

In [55]:
training_df = training_df.drop(columns=['sr_type_desc','sr_number'])

In [60]:
#training_df = training_df.drop(columns=[])

In [57]:
training_df = training_df.rename(columns={'sr_method_received_desc':'Report Method', "sr_location_zip_code":"Zip Code"})

In [58]:
training_df.head(10)

,Report Method,Zip Code,Request Year,Total Seconds,Response Time,Department
295290,Phone,78723,2019,152190.0,Slow Response,Austin Code
295291,Spot311 Interface,78744,2019,152934.0,Slow Response,Austin Code
295292,Web,78701,2019,113327.0,Slow Response,Public Works
295293,Spot311 Interface,78701,2019,121696.0,Slow Response,Austin Resource Recovery
295294,Phone,78701,2019,155392.0,Slow Response,Austin Code
295295,Phone,78745,2019,18479378.0,Slow Response,Animal Services
295296,Spot311 Interface,78745,2019,155819.0,Slow Response,Austin Code
295297,Phone,78746,2019,1097446.0,Slow Response,Austin Resource Recovery
295298,Spot311 Interface,78745,2019,117788.0,Slow Response,Animal Services
295299,Spot311 Interface,78660,2019,162582.0,Slow Response,Animal Services


In [59]:
#training_df.rename(columns={'index':'Index'}, inplace =True)
#command = 'Index serial primary key, Report Method text, Zip Code varchar(5), Request Year int, Total Seconds interval, Response Time text, Department text'


training_df.to_sql('database', con=engine, if_exists='replace')

engine.execute("SELECT * FROM database").fetchall()

[(295290, 'Phone', '78723', 2019, 152190.0, 'Slow Response', 'Austin Code'),
 (295291, 'Spot311 Interface', '78744', 2019, 152934.0, 'Slow Response', 'Austin Code'),
 (295292, 'Web', '78701', 2019, 113327.0, 'Slow Response', 'Public Works'),
 (295293, 'Spot311 Interface', '78701', 2019, 121696.00000000001, 'Slow Response', 'Austin Resource Recovery'),
 (295294, 'Phone', '78701', 2019, 155392.0, 'Slow Response', 'Austin Code'),
 (295295, 'Phone', '78745', 2019, 18479378.0, 'Slow Response', 'Animal Services'),
 (295296, 'Spot311 Interface', '78745', 2019, 155819.0, 'Slow Response', 'Austin Code'),
 (295297, 'Phone', '78746', 2019, 1097446.0, 'Slow Response', 'Austin Resource Recovery'),
 (295298, 'Spot311 Interface', '78745', 2019, 117788.00000000001, 'Slow Response', 'Animal Services'),
 (295299, 'Spot311 Interface', '78660', 2019, 162582.0, 'Slow Response', 'Animal Services'),
 (295300, 'Spot311 Interface', '78757', 2019, 162712.0, 'Slow Response', 'Animal Services'),
 (295301, 'Phone'